# 🔥 FILE 2-B: Optimizer, Activation & Regularization

**PHẦN 2 - INTERMEDIATE (CORE DEEP LEARNING)**

---

## 📋 Nội Dung

✅ Activation functions nâng cao (ReLU, LeakyReLU, GELU, Swish)

✅ Optimizers chi tiết (SGD, Momentum, Adam, AdamW)

✅ Learning rate và ảnh hưởng

✅ Learning rate scheduling

✅ Regularization techniques:
- Dropout
- Weight Decay (L2)
- Batch Normalization

✅ So sánh và thực nghiệm

---

## ⏱️ Thời Gian Học: 2.5-3 giờ

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

---

# 1️⃣ Activation Functions Nâng Cao

## So Sánh Các Activations

In [ ]:
# Visualize activations
x = torch.linspace(-3, 3, 200)

activations = {
    'ReLU': nn.ReLU()(x),
    'LeakyReLU': nn.LeakyReLU(0.1)(x),
    'ELU': nn.ELU()(x),
    'GELU': nn.GELU()(x),
    'Swish (SiLU)': nn.SiLU()(x),
    'Tanh': nn.Tanh()(x),
}

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, (name, y) in enumerate(activations.items()):
    axes[i].plot(x.numpy(), y.numpy(), linewidth=2)
    axes[i].set_title(name, fontsize=12, fontweight='bold')
    axes[i].grid(True, alpha=0.3)
    axes[i].axhline(y=0, color='k', linestyle='--', alpha=0.3)
    axes[i].axvline(x=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print("""
ĐÁNH GIÁ:

ReLU: max(0, x)
  ✅ Nhanh, đơn giản, phổ biến nhất
  ❌ Dying ReLU problem
  
LeakyReLU: max(0.1x, x)
  ✅ Fix dying ReLU
  ❌ Cần tune alpha
  
GELU: x * Φ(x)  [Gaussian Error Linear Unit]
  ✅ Smooth, dùng trong Transformers (BERT, GPT)
  ❌ Chậm hơn ReLU
  
Swish/SiLU: x * sigmoid(x)
  ✅ Smooth, self-gated
  ✅ Tốt cho deep networks
  
KHUYẾN NGHỊ:
  - Default: ReLU
  - Transformers: GELU
  - Deep networks: Swish/SiLU
  - Dying ReLU problem: LeakyReLU
""")

---

# 2️⃣ Optimizers Chi Tiết

## SGD vs Momentum vs Adam

In [ ]:
print("=" * 70)
print("OPTIMIZER COMPARISON")
print("=" * 70)

# Prepare data
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train = torch.FloatTensor(X[:800])
y_train = torch.LongTensor(y[:800])
X_val = torch.FloatTensor(X[800:])
y_val = torch.LongTensor(y[800:])

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Model
def create_model():
    return nn.Sequential(
        nn.Linear(20, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 2)
    )

# Test different optimizers
def train_with_optimizer(optimizer_fn, name, epochs=50):
    model = create_model().to(device)
    optimizer = optimizer_fn(model.parameters())
    criterion = nn.CrossEntropyLoss()
    
    losses = []
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Validation loss
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val.to(device))
            val_loss = criterion(val_outputs, y_val.to(device))
            losses.append(val_loss.item())
    
    return losses

# Compare optimizers
optimizers = {
    'SGD': lambda p: optim.SGD(p, lr=0.01),
    'SGD+Momentum': lambda p: optim.SGD(p, lr=0.01, momentum=0.9),
    'Adam': lambda p: optim.Adam(p, lr=0.001),
    'AdamW': lambda p: optim.AdamW(p, lr=0.001, weight_decay=0.01),
}

results = {}
for name, opt_fn in optimizers.items():
    print(f"Training with {name}...")
    results[name] = train_with_optimizer(opt_fn, name)

# Plot comparison
plt.figure(figsize=(12, 6))
for name, losses in results.items():
    plt.plot(losses, label=name, linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.title('Optimizer Comparison', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nFinal losses:")
for name, losses in results.items():
    print(f"  {name:15s}: {losses[-1]:.4f}")

## Optimizer Details

In [ ]:
print("""
=" * 70
CHI TIẾT CÁC OPTIMIZER
=" * 70

1. SGD (Stochastic Gradient Descent)
   Formula: θ = θ - lr * ∇L
   
   ✅ Đơn giản, ổn định
   ❌ Chậm, dễ stuck local minima
   
2. SGD + Momentum
   Formula: 
     v = β*v + ∇L
     θ = θ - lr * v
   
   ✅ Tăng tốc, vượt local minima
   ✅ Thường β = 0.9
   
3. Adam (Adaptive Moment Estimation)
   Combines:
     - Momentum (first moment)
     - RMSprop (second moment)
   
   ✅ Tự động điều chỉnh LR cho từng parameter
   ✅ Hội tụ nhanh
   ✅ PHỔ BIẾN NHẤT
   
   Hyperparameters:
     - lr: 0.001 (default)
     - betas: (0.9, 0.999)
     - eps: 1e-8
   
4. AdamW (Adam with Weight Decay)
   = Adam + L2 regularization ĐÚNG CÁCH
   
   ✅ Fix weight decay bug trong Adam
   ✅ Tốt hơn Adam cho nhiều tasks
   ✅ Dùng trong Transformers
   
   weight_decay: 0.01 (typical)

KHUYẾN NGHỊ:
  - Bắt đầu: Adam (lr=0.001)
  - Fine-tuning: SGD + Momentum
  - Modern: AdamW
  - Transformers: AdamW + LR scheduling
""")

---

# 3️⃣ Learning Rate

## LR Experiment

In [ ]:
print("=" * 70)
print("LEARNING RATE EXPERIMENT")
print("=" * 70)

learning_rates = [0.0001, 0.001, 0.01, 0.1, 1.0]
lr_results = {}

for lr in learning_rates:
    print(f"Testing LR={lr}...")
    model = create_model().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    losses = []
    for epoch in range(50):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(X_val.to(device)), y_val.to(device))
            losses.append(val_loss.item())
    
    lr_results[lr] = losses

# Plot
plt.figure(figsize=(12, 6))
for lr, losses in lr_results.items():
    plt.plot(losses, label=f'LR={lr}', linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Rate Comparison', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print("""
KẾT LUẬN:
  - LR quá nhỏ (0.0001): Học chậm
  - LR vừa (0.001, 0.01): Tốt nhất
  - LR quá lớn (0.1, 1.0): Không hội tụ
""")

## Learning Rate Scheduler

In [ ]:
print("=" * 70)
print("LEARNING RATE SCHEDULING")
print("=" * 70)

model = create_model().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Start high

# StepLR: Giảm LR mỗi N epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

lrs = []
for epoch in range(50):
    # Training code here...
    lrs.append(optimizer.param_groups[0]['lr'])
    scheduler.step()  # Update LR

plt.figure(figsize=(10, 5))
plt.plot(lrs, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('StepLR Scheduler', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

print("""
CÁC SCHEDULER PHỔ BIẾN:

1. StepLR
   lr = lr * gamma every step_size epochs
   
2. ReduceLROnPlateau
   Giảm LR khi val loss không cải thiện
   
3. CosineAnnealingLR
   LR theo cosine curve
   
4. OneCycleLR
   Tăng rồi giảm LR (super-convergence)

USAGE:
  scheduler = optim.lr_scheduler.StepLR(...)
  
  for epoch in range(epochs):
      train(...)  
      scheduler.step()  # ← After each epoch
""")

---

# 4️⃣ Regularization

## Dropout

In [ ]:
print("=" * 70)
print("DROPOUT")
print("=" * 70)

class ModelWithDropout(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        self.fc1 = nn.Linear(20, 64)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(32, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)  # Drop neurons
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

model = ModelWithDropout(dropout_rate=0.5)

print("""
DROPOUT HOẠT ĐỘNG:

Training mode:
  - Randomly "drop" p% neurons (set to 0)
  - Prevents co-adaptation
  - Acts like ensemble

Eval mode:
  - Use ALL neurons
  - Scale by (1-p)

LỢI ÍCH:
  ✅ Giảm overfitting
  ✅ Đơn giản, hiệu quả
  ✅ Ensemble effect

DROPOUT RATE:
  - Typical: 0.2 - 0.5
  - Input layer: 0.1 - 0.2
  - Hidden layers: 0.5

⚠️ QUAN TRỌNG:
  model.train()  # Enable dropout
  model.eval()   # Disable dropout
""")

## Weight Decay (L2 Regularization)

In [ ]:
print("=" * 70)
print("WEIGHT DECAY")
print("=" * 70)

# Without weight decay
optimizer_no_wd = optim.Adam(model.parameters(), lr=0.001)

# With weight decay
optimizer_wd = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

print("""
WEIGHT DECAY:

Formula: L_total = L + λ * ||w||²

Ý nghĩa:
  - Penalize large weights
  - Prefer smaller weights
  - Prevent overfitting

CÁCH DÙNG:
  optimizer = optim.AdamW(
      model.parameters(),
      lr=0.001,
      weight_decay=0.01  # λ
  )

GIÁ TRỊ:
  - Typical: 0.0001 - 0.01
  - Lớn hơn = regularization mạnh hơn

Adam vs AdamW:
  - Adam: Weight decay implementation có bug
  - AdamW: Correct implementation
  - Khuyến nghị: Dùng AdamW
""")

## Batch Normalization

In [ ]:
print("=" * 70)
print("BATCH NORMALIZATION")
print("=" * 70)

class ModelWithBatchNorm(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(20, 64)
        self.bn1 = nn.BatchNorm1d(64)  # BatchNorm after Linear
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.fc3 = nn.Linear(32, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)      # Normalize
        x = torch.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x

print("""
BATCH NORMALIZATION:

Công thức:
  x_norm = (x - mean) / sqrt(var + eps)
  y = γ * x_norm + β

LỢI ÍCH:
  ✅ Tăng tốc training
  ✅ Cho phép LR cao hơn
  ✅ Giảm sensitivity to initialization
  ✅ Regularization effect

VỊ TRÍ:
  Linear → BatchNorm → Activation
  HOẶC
  Linear → Activation → BatchNorm

TYPES:
  - BatchNorm1d: For FC layers
  - BatchNorm2d: For Conv layers
  - BatchNorm3d: For 3D Conv

⚠️ LƯU Ý:
  - Khác nhau giữa train/eval mode
  - model.train() / model.eval()
""")

---

# ✅ Tổng Kết FILE 2-B

## Học Được

✅ **Activations**: ReLU, LeakyReLU, GELU, Swish

✅ **Optimizers**: SGD, Momentum, Adam, AdamW

✅ **Learning Rate**: Experiments, Scheduling

✅ **Regularization**: Dropout, Weight Decay, BatchNorm

## Quick Reference

```python
# Model with regularization
class RegularizedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(20, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        return x

# Optimizer with weight decay
optimizer = optim.AdamW(model.parameters(), 
                        lr=0.001, 
                        weight_decay=0.01)

# LR Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size=10, 
                                      gamma=0.5)
```

---